In [1]:
using BenchmarkTools, SpecialFunctions
using LinearAlgebra, Statistics
using Plots
using Random, Distributions
gr(fmt=:png); # setting for easier display in jupyter notebooks

**Introduction**

I'm teaching myself ``julia``. The simplest way for me todo so is to work with a model that I know extremely well---the Ricardian model of Eaton and Kortum (2002). So below I will outline the economic environment and then use the properties of ``julia`` to map the fundamental, mathematical environment into a computational/quantitative outcome. 

Outside of performance properties, the more I reflect on this, this is a huge appeal of the ``julia`` language. We can express the code and design it in a way that most closely corresponds with the math/model. In contrast, in for example MATLAB, where the map from white board to computation is big.

---

**Countries.** To keep things simple, let's first work with a simple two country set up---a home and a foreign country, $h$, $f$.

**Individual Goods Production.**
Within a country, there is a continuum of intermediate goods indexed by $\omega \in [0, 1]$. As in the Ricardian model of \citet{dornbusch1977comparative} and \citet{eaton2002technology}, intermediate goods are not nationally differentiated. Thus, intermediate $\omega$ produced in one country is a perfect substitute for the same intermediate $\omega$ produced by another country. Competitive firms produce intermediate goods with linear production technologies:

\begin{align}
q(\omega) = z(\omega) \ell,
\end{align}

where $z$ is the productivity level of firms and $\ell$ is the number of efficiency units of labor. To ship a technology to another country, iceberg costs are incurred. I will write this as $\tau_{hf}$ which is the value the home country must incur to buy from the foreign country. The notation here will correspond with the row, column format of a matrix, so for a given row all the trade costs across the different columns are the ones that they face to buy from different countries. Then the usual normalization here is that $\tau_{hh} = 1$. Again, on the computer, this means that the diagonal entries are one.

**The Aggregator.** In this model, intermediate goods are aggregated by a competitive final-goods producer. In Julia, what would be interesting is to make the aggregator generic and let the user specify how this works. But for know, we will follow the canonical situation with  who has a standard CES production function:

\begin{align}
Q = \left[ \int_0^1 q(\omega)^{\frac{\sigma - 1}{\sigma}} d\omega \right]^{\frac{\sigma}{\sigma - 1}},
\label{eq:ces}
\end{align}

where $q(\omega)$ is the quantity of individual intermediate goods $\omega$ demanded by the final-goods firm, and $\sigma$ is elasticity of substitution across variety. I want to talk through some properties/definitions related to this aggregator.

- The aggregator in (\ref{eq:ces}) gives rise to the following the demand curve (in quantities) for an individual variety:

\begin{align}
q(\omega) & = \left(\frac{p_h(\omega)}{P_h}\right)^{-\sigma}Q.
\label{eq:demand_curve}
\end{align}

where $Q$ is the aggregate demand for the final good; $P_h$ is the CES price index, defined below; And $p_h(\omega)$ is the price paid by the home country for good $\omega$ (which is not the same necessarily as the price offered by a home country supplier.

- The demand curve can also be expressed in shares, not quantities. So

\begin{align}
\frac{p_h(\omega)q(\omega)}{P_h Q} & = \left(\frac{p_h(\omega)}{P_h}\right)^{1-\sigma}.
\label{eq:demand_curve_share}
\end{align}

which says that the value of of good $\omega$ relative to total expenditures $P_h Q$ is given by the formula above. This helps as we can compute expenditure patterns while being agnostic about the actual size of the economy, i.e. $Q$. One useful relationship that builds on this is to condition on the source and aggregate to arrive at the share of goods purchased from another country. How does this work? 

\begin{align}
\pi_{h,f} = \int_{p_h = p_{h,f}} \left(\frac{p_h(\omega)}{P_h}\right)^{1-\sigma} d\omega
\end{align}

Where the notation here $\pi_{h,f}$ is the share of goods that the home country sources from the foreign country. This is equal do the integral over all goods purchased from them, hence, the conditioning where $p_h = p_{h,f}$ and then the inside of the brackets is the same as above. In the computer, the code will be setup with the same row, column notation. So fix a row, then as you go across the different columns you'll view the share of expenditures the, say home country, coming from other countries. 

A small side note: If you stare at this enough (and know the liturature), than you can start to see a gravity like relationship here without even imposing the Frechet assumption. The key issue essentially is the conditional distribution of goods prices and how it relates to the aggregate, home CES price index discussed next.

- The **CES Price Index** is given by

\begin{align}
P_h  = \left [ \int_{0}^{1} p_h(\omega)^{1-\sigma} d\omega \right]^{\frac{1}{1-\sigma}}
\label{eq:ces_price_index}
\end{align}

which takes all the prices at the micro-level and the is like a harmonic mean of these prices. Also note how one can substitute \ref{eq:ces_price_index} into the share formula to get an expression of expenditures shares purely in terms of micro-level prices.

**The Technology Distribution** 

This is the key component that makes Eaton and Kortum fly. First, let's be a bit more general here. One could simply think of each countries **individual productivity** draws being characterized by some distribution:

\begin{align}
Z_h \sim F_{h}(z)
\end{align}

where $F_{h}$ is a well defined cumulative probability distribution with the associated pdf $f_h(z)$. What this means is for each $\omega$ variety, in say the home country, the technology/productivity to produce that good is a random $z$ which is characterized by the distribution above. As I'll talk about below, what this allows one to think about is (i) if you hand me a distribution from which I can simulate from (ii) then with assumptions on market structure I can construct prices and (iii) with prices I can construct trade flows.

Now Eaton and Kortum make a very specific distributional assumption. They use a Type II extreme value distribution also known as a Frechet:

\begin{align}
Z_i \sim \exp\left(-T_i z^{-\theta}\right)
\end{align}

where the distribution is characterized by a centering parameter $T_i$ which is allowed to vary across countries and a shape parameter $\theta$ which is assumed to be constant across countries. These assumptions are critical to the closed form characterizations in Eaton and Kortum. However, return to the discussion above. On the computer..there are only two limiting factors (i) can you simulate from the distribution and (ii) the empirical question of how to discipline the distribution from which you are simulating. 

**Optimization** 

I'm going to walk through a couple of observations about how this economy works.

The first observation is that within a country, the key equilibrium price is the wage rate $w$. This wage is the same independent of the labor market / goods-producing sector / etc. This derives from the (within-country) free mobility of labor assumption (which I did not talk about). In some other notebooks, I'll relax this. But for now, labor is freely mobile and now we just need to find the wage so that labor demand equals labor supply. 

The second observation is that given competitive firms and that they face the wage rate $w$, we know that their marginal cost (and hence price at which they are willing to sell the good domestically) must be:

\begin{align}
p_{h,h}(\omega) = \frac{w_{h}}{z_{h}(\omega)},
\label{eq:ek_wage}
\end{align}

and then the price at which they would be willing to export the good, say from home to foreign is just adjusted by the ice-berg trade costs so 

\begin{align}
p_{f,h}(\omega) = \frac{\tau_{f,h} w_{h}}{z_{h}(\omega)}.
\end{align}

The third question is who buys what from whom? This aspect of the model is that really is just an application of the min operator. So buy the good from the country with the lowest price:

\begin{align}
p_{h}(\omega) = \min\left\{ \frac{w_{h}}{z_{h}(\omega)}, \ \frac{\tau_{h,f} w_{f}}{z_{f}(\omega)} \right\}
\end{align}

where the thing on the left is the domestic price, and then the thing on the right is on the world price (inclusive of the trade cost).

**Computing**

First, I'm going to compute this via simulation. The idea will be to think of the product space $\omega$ as an integer on the real line which indexes the goods location. Then for each of those products, each country is randomly assigned a productivity from the distribution. Then the number of goods, so to speak, is set to be large and thus will approximate the integrals above. 

Given everything laid out, some wages $w_i$ and then primitives regarding technologies $T$, $\theta$, $\sigma$ and $\tau$ we can directly compute the pattern of trade. This is what I do below which is comprised of several steps:

- The CES function ``ces_shares`` that takes prices and sourcing decisions to compute trade shares.

- A function that computes potential prices based on marginal costs, ``marginal_costs``

- A function that constructs a vector of productivity for each good, across all countries, given the Frechet distribution, this is the ``make_productivity`` function. Note, future, it would be nice if the distribution type could be directly passed into this and also appropriately indexed. Maybe with a dictionary?

In [2]:
function ces_shares(p::Matrix{Float64}, source_country::Matrix{CartesianIndex{2}}, σ::Float64)
    # this will take in (i) prices purchased at and
    # a cartesian index that can determine country supplier
    # then a parameter for the CES aggregator.
    # I'm toying around with explicity typing here. Any thoughts?
    # When presented this way. One can see how any arbitrary aggregator could be put in place.
    
    ngoods = length(p) # number of goods
    
    ncntry = ndims(source_country) # number of coutnries
    
    price_index = ( sum(p.^(1-σ)) / ngoods )^(1/(1-σ)) # CES price index
    
    # now construct the shares...
    trade_shares = zeros(ncntry)
    
    source_prob = zeros(ncntry)
    
    for cntry = 1:ncntry
        # This is the loop that conditions on the source country
        # first fix a country...

        supplier = [ zzz[2] == cntry for zzz in source_country ]
        # this is then a simple inline expression saying, for each 
        # index in the source country, which onex zzz[2] are equall 
        # to the country of origin...
        
        source_prob[cntry] = sum(supplier) / ngoods
        # This will compute the simple probability of being the low cost 
        # supplier. In the Frechet model = trade share. Alternative distributional
        # assumptions will deliver something different.
        
        trade_shares[cntry] = sum((p[supplier]./price_index).^(1-σ)) / ngoods
        
    end
        
    return price_index, trade_shares, source_prob

end

ces_shares (generic function with 1 method)

In [3]:
function marginal_cost(z::AbstractArray, wage::AbstractArray, τ::AbstractArray)
    return (τ.*wage)./z
end

marginal_cost (generic function with 1 method)

In [110]:
function make_productity(T::Matrix{Float64},θ::Float64,ngoods::Int64)
    
    seed = 3281978
    
    ncntry = length(T)
    
    z = zeros(ngoods,ncntry)
    
    for cntry = 1:ncntry
    
        dist = Frechet(θ, T[cntry]^(1/θ))
        # Ideally, it would be cool to potentially be able to pass as an argument
        # the distibution for each country or something like that.
    
        z[:,cntry] = rand(MersenneTwister(seed + cntry), dist, ngoods, 1);
        #I tried the rand! command (if modifies the argument),
        # it did not work...seemed to rewrite everytime
        # this thing looped through.
    
    end
    
    return z
    
end

make_productity (generic function with 2 methods)

In [117]:
tau_matrix = [1 1.5; 1.5 1]
T = [1.0 2.0]
w = [1.0 1.0]
θ = 4.0

ncntry = ndims(tau_matrix)
ngoods = 100000

z = make_productity(T,θ,ngoods)
# this is the slow part of the code, not the min part...

price_index = zeros(ncntry)
shares = zeros(ncntry,ncntry)
source_prob = zeros(ncntry,ncntry)

for cntry = 1:ncntry

    p, source_country = findmin(marginal_cost(z, w, tau_matrix[cntry,:]'); dims = 2);
    # find the min over marginal costs
    # note that the tau_matrix returns a vector, not a row vector as I was anticipating
    # so take the transpose to make it work.

    (price_index[cntry], shares[cntry,:], source_prob[cntry,:]) = ces_shares(p, source_country, 2.0);
    # then this is set up so the row is the buyer...then you go across the columns
    # and that is share of goods the home country buys from that country.
    
end

In [118]:
shares

2×2 Matrix{Float64}:
 0.717703   0.282297
 0.0899972  0.910003

In [63]:
source_prob

2×2 Matrix{Float64}:
 0.717294  0.282706
 0.090292  0.909708

In [104]:
tau_matrix = [1 1.5; 1.5 1]
T = [1.0 2.0]
w = [1.0 1.0]
θ = 4.0

ncntry = ndims(tau_matrix)
ngoods = 1000000

z = make_productity(T,θ, ngoods);

In [103]:
z = make_productity(T,θ, ngoods);